In [36]:
from ngsolve import *
import netgen.gui
import time

## 2维曲面生成

In [38]:
from netgen.csg import Pnt,SplineCurve2d,CSGeometry,Revolution,Sphere
import numpy as np
import netgen.meshing as ngm
from netgen.csg import *
from netgen.meshing import MeshingStep

In [75]:
geo = CSGeometry()
geo.Add(Sphere(Pnt(0,0,0),1))
mesh = Mesh(geo.GenerateMesh(maxh=0.5, perfstepsend=MeshingStep.MESHSURFACE))
Draw(mesh)

 Start Findpoints
 Analyze spec points
 Find edges
 Start Findpoints
 Analyze spec points
 Find edges
 Start Findpoints
 Analyze spec points
 Find edges
 Surface 1 / 1
 Optimize Surface


## 有限元空间
* 标量空间 for multiplier
* 向量空间 for position and velocity

In [76]:
fes = H1(mesh,order=1)
fesV = VectorH1(mesh,order=1)
fesMix = fes*fesV

## 有限元函数
* 位置 X
* 位移 Disp

In [77]:
Disp = GridFunction(fesV)
X_old = GridFunction(fesV)
Solution = GridFunction(fesMix)

## 设定位置X的初值

In [78]:
Vertices_Coords = np.array([v.point for v in mesh.vertices])
X_old.vec.data = BaseVector(Vertices_Coords.flatten('F'))

## Mass lumping on surface

In [79]:
ir = IntegrationRule(points = [(0,0), (1,0), (0,1)], weights = [1/6, 1/6, 1/6])
ds_lumping = ds(intrules = { TRIG : ir })

## Time stepping parameter

In [80]:
tauval = 0.02
t_old = 0

## 显示网格移动的设定

In [71]:
SetVisualization(deformation=True)

In [72]:
Draw(Disp,mesh,'disp',deformation=True)

## BGN弱形式
* BGN本身求解的是新时刻的X；这里求解的是D = X^n+1 - X^n

In [81]:
kappa, D = fesMix.TrialFunction()
chi, eta = fesMix.TestFunction()

lhs = BilinearForm(fesMix)
lhs += InnerProduct(D,specialcf.normal(3))*chi*ds_lumping + tauval*kappa*chi*ds_lumping
lhs += -InnerProduct(eta,specialcf.normal(3))*kappa*ds_lumping + InnerProduct(grad(D).Trace(),grad(eta).Trace())*ds

rhs = LinearForm(fesMix)
rhs += -InnerProduct(grad(X_old).Trace(),grad(eta).Trace())*ds

## 时间演化

In [82]:
t_old = 0
while t_old<10:
    mesh.SetDeformation(Disp)
    lhs.Assemble()
    rhs.Assemble()
    Solution.vec.data = lhs.mat.Inverse(inverse="pardiso")*rhs.vec
    
    Disp.vec.data = BaseVector(Disp.vec.FV().NumPy() + tauval*Solution.components[1].vec.FV().NumPy())
    X_old.vec.data = BaseVector(X_old.vec.FV().NumPy() + tauval*Solution.components[1].vec.FV().NumPy())
    Redraw()
    time.sleep(0.01)
    t_old += tauval